In [15]:
# Dependencies
from TwitterAPI import TwitterAPI
import json
import pandas as pd
import urllib.parse

In [2]:
# Login using credentials stored in auth.json file

# Define authentication file path
auth_path = 'data/auth.json'

# Read auth.json file
with open(auth_path, 'r') as file:
    auth_dict = json.load(file)

# Execute login
api = TwitterAPI(**auth_dict)

# Verify credentials
req = api.request('account/verify_credentials')
assert req.status_code == 200, 'Authentication failed!'

In [3]:
# Get current application rate limit
req = api.request('application/rate_limit_status')
res = req.json()  # Save response dictionary

# Get resources
resources = res['resources']

# Show search endopoint rate limit
print('Current endpoint: {:s}'.format(str(resources['application']['/application/rate_limit_status'])))

# Show search endpoint rate limit
print('Search endpoint: {:s}'.format(str(resources['search']['/search/tweets'])))

Current endpoint: {'limit': 180, 'remaining': 179, 'reset': 1577220966}
Search endpoint: {'limit': 180, 'remaining': 180, 'reset': 1577220966}


In [4]:
# Return tweets for a some given users in the last few days

# Get file containing users screen names
with open('data/selected_users.json', 'r') as file:
    screen_names = json.load(file)

# Show retrieved screen names
print(screen_names)

['FeminismInIndia', 'EvrydayFeminism', 'UN_Women', 'wwd', 'womensmarch', 'womensaid', 'WomensInstitute', 'womensmediacntr', 'womenmakemovies', 'womenslink', 'NationalNOW', 'blkwomenshealth', 'IndiaMeToo']


In [5]:
# Retrieve users from screen names
res = api.request('users/lookup', {
    'screen_name': ','.join(screen_names),
    'include_entities': False
})
res_status = res.status_code

# Print returned status code
print('Returned status code: {:d}'.format(res_status))
print()

# Get users
users = [x for x in res]

# Show first user object
print(users[0])
print()

Returned status code: 200

{'id': 2247560024, 'id_str': '2247560024', 'name': 'Feminism in India', 'screen_name': 'FeminismInIndia', 'location': 'India', 'description': 'FII amplifies the voices of women and marginalized groups using tools of art, media, culture, technology & community. Subscribe: \nhttps://t.co/wMm4oPtpiT', 'url': 'http://t.co/92ddyjkbEa', 'entities': {'url': {'urls': [{'url': 'http://t.co/92ddyjkbEa', 'expanded_url': 'http://feminisminindia.com', 'display_url': 'feminisminindia.com', 'indices': [0, 22]}]}, 'description': {'urls': [{'url': 'https://t.co/wMm4oPtpiT', 'expanded_url': 'http://eepurl.com/cjuLbv', 'display_url': 'eepurl.com/cjuLbv', 'indices': [130, 153]}]}}, 'protected': False, 'followers_count': 36210, 'friends_count': 1728, 'listed_count': 406, 'created_at': 'Sun Dec 15 18:02:24 +0000 2013', 'favourites_count': 3090, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 49728, 'lang': None, 'status': {'created_a

In [6]:
# Define a subset of attributes to be kept for each user
kept_attr = ('id_str', 'name', 'screen_name', 'description', 'location', 'followers_count', 
           'friends_count', 'verified', 'statuses_count')

# Keep only a subset of user's attributes
for i, user in enumerate(users):
    users[i] = {k: user[k]for k in kept_attr}
    
# Show first user object
print(users[0])

{'id_str': '2247560024', 'name': 'Feminism in India', 'screen_name': 'FeminismInIndia', 'description': 'FII amplifies the voices of women and marginalized groups using tools of art, media, culture, technology & community. Subscribe: \nhttps://t.co/wMm4oPtpiT', 'location': 'India', 'followers_count': 36210, 'friends_count': 1728, 'verified': True, 'statuses_count': 49728}


In [7]:
# Store users data in a tabular form through pandas DataFrame object
users = pd.DataFrame(users)
users.head()

,id_str,name,screen_name,description,location,followers_count,friends_count,verified,statuses_count
0,2247560024,Feminism in India,FeminismInIndia,FII amplifies the voices of women and marginal...,India,36210,1728,True,49728
1,591920938,Everyday Feminism,EvrydayFeminism,Intersectional feminism for your everyday life...,,127136,1103,True,19039
2,17137628,UN Women,UN_Women,UN Women is the UN entity for gender equality ...,Worldwide,1716993,4058,True,57563
3,19784831,WWD,wwd,Fashion. Beauty. Business.,,2870176,475,True,82633
4,800942537083068416,Women's March,womensmarch,"Three years of resistance, three years of trai...","Washington, D.C.",636422,763,True,26992


In [11]:
# Save users table as .csv file
users.to_csv('data/database/users.csv')